# Modal Dialog

> Pattern for modal dialogs with customizable content, sizes, and actions

In [ ]:
#| default_exp patterns.modal_dialog

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
from typing import Optional, Any, Union, List
from enum import Enum
from fasthtml.common import *

from cjm_fasthtml_interactions.core.html_ids import InteractionHtmlIds
from cjm_fasthtml_daisyui.components.actions.modal import modal, modal_box, modal_backdrop
from cjm_fasthtml_daisyui.components.actions.button import btn, btn_sizes, btn_styles, btn_modifiers
from cjm_fasthtml_tailwind.utilities.spacing import p
from cjm_fasthtml_tailwind.utilities.sizing import w, h, max_w
from cjm_fasthtml_tailwind.utilities.layout import position, right, top
from cjm_fasthtml_tailwind.core.base import combine_classes

## Modal Size Options

The `ModalSize` enum defines predefined size options for modal dialogs.

In [ ]:
#| export
class ModalSize(Enum):
    """Predefined size options for modal dialogs."""
    SMALL = "sm"      # max-w-md (28rem / 448px)
    MEDIUM = "md"     # max-w-lg (32rem / 512px)
    LARGE = "lg"      # max-w-2xl (42rem / 672px)
    XLARGE = "xl"     # max-w-4xl (56rem / 896px)
    FULL = "full"     # w-11/12 h-11/12 (near full screen)
    CUSTOM = "custom" # Use custom width/height

## ModalDialog Function

The `ModalDialog` function creates a modal dialog component using DaisyUI's modal system.

Key features:
- Native HTML `<dialog>` element with DaisyUI styling
- Customizable sizes (small, medium, large, xlarge, full, or custom)
- Optional close button (top-right X button)
- Optional backdrop for closing on outside click
- Support for HTMX updates to modal content
- Programmatic show/close via JavaScript API
- Auto-show support for immediate display
- Flexible content area with optional styling

In [ ]:
#| export
def ModalDialog(
    modal_id: str,  # Unique identifier for the modal
    content: Any,  # Content to display in the modal
    size: Union[ModalSize, str] = ModalSize.MEDIUM,  # Size preset or custom size
    show_close_button: bool = True,  # Whether to show X close button in top-right
    close_on_backdrop: bool = True,  # Whether clicking backdrop closes modal
    auto_show: bool = False,  # Whether to show modal immediately on render
    content_id: Optional[str] = None,  # Optional ID for content area (for HTMX targeting)
    custom_width: Optional[str] = None,  # Custom width class (e.g., "w-96")
    custom_height: Optional[str] = None,  # Custom height class (e.g., "h-screen")
    box_cls: Optional[str] = None,  # Additional classes for modal box
    **kwargs  # Additional attributes for the dialog element
) -> FT:  # Dialog element with modal dialog configured
    """Create a modal dialog using native HTML dialog element with DaisyUI styling."""
    # Generate modal dialog ID
    dialog_id = InteractionHtmlIds.modal_dialog(modal_id)
    
    # Generate content ID if not provided
    if content_id is None:
        content_id = InteractionHtmlIds.modal_dialog_content(modal_id)
    
    # Build modal box size classes
    size_classes = []
    
    # Handle size parameter
    if isinstance(size, ModalSize):
        if size == ModalSize.SMALL:
            size_classes.extend([max_w.md])
        elif size == ModalSize.MEDIUM:
            size_classes.extend([max_w.lg])
        elif size == ModalSize.LARGE:
            size_classes.extend([max_w._2xl])
        elif size == ModalSize.XLARGE:
            size_classes.extend([max_w._4xl])
        elif size == ModalSize.FULL:
            size_classes.extend([w("11/12"), h("11/12")])
        elif size == ModalSize.CUSTOM:
            if custom_width:
                size_classes.append(custom_width)
            if custom_height:
                size_classes.append(custom_height)
    else:
        # If size is a string, treat it as a custom class
        size_classes.append(size)
    
    # Build modal box classes
    box_classes = [modal_box] + size_classes
    if box_cls:
        box_classes.append(box_cls)
    
    # Build modal content
    modal_content_parts = []
    
    # Add close button if requested
    if show_close_button:
        close_btn = Form(
            Button(
                "✕",
                cls=combine_classes(
                    btn,
                    btn_sizes.sm,
                    btn_modifiers.circle,
                    btn_styles.ghost,
                    position.absolute,
                    right(2),
                    top(2)
                )
            ),
            method="dialog"
        )
        modal_content_parts.append(close_btn)
    
    # Add main content with optional ID
    if isinstance(content, (list, tuple)):
        # If content is a list/tuple, wrap it
        content_wrapper = Div(*content, id=content_id)
    elif hasattr(content, 'attrs') and 'id' in content.attrs:
        # Content already has an ID, use it as-is
        content_wrapper = content
    else:
        # Wrap content with ID
        content_wrapper = Div(content, id=content_id)
    
    modal_content_parts.append(content_wrapper)
    
    # Build modal box
    modal_box_element = Div(
        *modal_content_parts,
        cls=combine_classes(*box_classes)
    )
    
    # Build dialog parts
    dialog_parts = [modal_box_element]
    
    # Add backdrop if requested
    if close_on_backdrop:
        backdrop = Form(
            Button("close"),
            method="dialog",
            cls=str(modal_backdrop)
        )
        dialog_parts.append(backdrop)
    
    # Create the dialog element
    dialog = Dialog(
        *dialog_parts,
        id=dialog_id,
        cls=str(modal),
        **kwargs
    )
    
    # Add auto-show script if requested
    if auto_show:
        show_script = Script(f"""
            (function() {{
                const modal = document.getElementById('{dialog_id}');
                if (modal && typeof modal.showModal === 'function') {{
                    // Use requestAnimationFrame to ensure DOM is ready
                    requestAnimationFrame(() => modal.showModal());
                }}
            }})();
        """)
        return Div(dialog, show_script)
    
    return dialog

## Trigger Button Helper

Helper function to create a button that opens a modal dialog.

In [ ]:
#| export
def ModalTriggerButton(
    modal_id: str,  # ID of the modal to trigger
    label: str,  # Button label text
    button_cls: Optional[str] = None,  # Additional button classes
    **kwargs  # Additional button attributes
) -> FT:  # Button element that triggers modal
    """Create a button that opens a modal dialog."""
    dialog_id = InteractionHtmlIds.modal_dialog(modal_id)
    
    button_classes = [btn]
    if button_cls:
        button_classes.append(button_cls)
    
    return Button(
        label,
        onclick=f"document.getElementById('{dialog_id}').showModal()",
        cls=combine_classes(*button_classes) if len(button_classes) > 1 else str(btn),
        **kwargs
    )

## Usage Examples

Here are complete examples showing different modal use cases:

In [ ]:
from cjm_fasthtml_daisyui.components.data_display.card import card_body, card_title
from cjm_fasthtml_daisyui.components.actions.button import btn_colors

# Example 1: Simple info modal
simple_modal = ModalDialog(
    modal_id="info",
    content=Div(
        H2("Information", cls=str(card_title)),
        P("This is a simple informational modal."),
        cls=str(card_body)
    ),
    size=ModalSize.SMALL
)

# Trigger button
trigger_btn = ModalTriggerButton(
    modal_id="info",
    label="Show Info",
    button_cls=str(btn_colors.info)
)

print("Example 1: Simple info modal")
print(f"Modal ID: {InteractionHtmlIds.modal_dialog('info')}")
print(f"Trigger: {trigger_btn}")

Example 1: Simple info modal
Modal ID: modal-info
Trigger: <button onclick="document.getElementById('modal-info').showModal()" class="btn btn-info">Show Info</button>


In [ ]:
# Example 2: Large modal with HTMX content loading
from cjm_fasthtml_interactions.patterns.async_loading import AsyncLoadingContainer, LoadingType

# Modal with async loading content
loading_modal = ModalDialog(
    modal_id="settings",
    content=AsyncLoadingContainer(
        container_id="settings-form",
        load_url="/api/settings/form",
        loading_type=LoadingType.SPINNER,
        loading_message="Loading settings..."
    ),
    size=ModalSize.LARGE,
    content_id="settings-form"
)

print("Example 2: Modal with async content")
print(f"Modal ID: {InteractionHtmlIds.modal_dialog('settings')}")
print(f"Content ID: {InteractionHtmlIds.modal_dialog_content('settings')}")

Example 2: Modal with async content
Modal ID: modal-settings
Content ID: modal-settings-content


In [ ]:
# Example 3: Full-screen modal for media viewing
fullscreen_modal = ModalDialog(
    modal_id="media-viewer",
    content=Div(
        Img(src="/images/large-photo.jpg", alt="Photo"),
        cls=combine_classes("flex", "items-center", "justify-center")
    ),
    size=ModalSize.FULL,
    show_close_button=True,
    close_on_backdrop=True
)

print("Example 3: Full-screen media viewer")
print(f"Modal ID: {InteractionHtmlIds.modal_dialog('media-viewer')}")

Example 3: Full-screen media viewer
Modal ID: modal-media-viewer


In [ ]:
# Example 4: Custom size modal with auto-show
custom_modal = ModalDialog(
    modal_id="welcome",
    content=Div(
        H2("Welcome!", cls=str(card_title)),
        P("Thanks for visiting our app."),
        cls=str(card_body)
    ),
    size=ModalSize.CUSTOM,
    custom_width="w-96",
    custom_height="h-48",
    auto_show=True
)

print("Example 4: Custom size with auto-show")
print(f"Modal ID: {InteractionHtmlIds.modal_dialog('welcome')}")

Example 4: Custom size with auto-show
Modal ID: modal-welcome


## Programmatic Control

You can show and close modals programmatically using JavaScript:

```javascript
// Show modal
document.getElementById('modal-my-modal').showModal();

// Close modal
document.getElementById('modal-my-modal').close();
```

Or in Python, using the helper to generate the correct JavaScript:

```python
# In your button onclick
Button(
    "Open Modal",
    onclick=f"document.getElementById('{InteractionHtmlIds.modal_dialog('my-modal')}').showModal()"
)

# Or use ModalTriggerButton helper
ModalTriggerButton(
    modal_id="my-modal",
    label="Open Modal"
)
```

## HTMX Integration

Modals work seamlessly with HTMX for dynamic content updates:

### Update Modal Content

Target the content area to update modal contents:

```python
# Create modal with content ID
modal = ModalDialog(
    modal_id="details",
    content=Div("Initial content", id="details-content"),
    content_id="details-content"
)

# Button that updates modal content and shows it
Button(
    "Load Details",
    hx_get="/api/details/123",
    hx_target=InteractionHtmlIds.as_selector(
        InteractionHtmlIds.modal_dialog_content("details")
    ),
    hx_swap="innerHTML",
    # Show modal after content loads
    hx_on__after_swap=f"document.getElementById('{InteractionHtmlIds.modal_dialog('details')}').showModal()"
)
```

### Close Modal After Action

Close modal after a successful form submission:

```python
# Form inside modal
Form(
    # ... form fields ...
    Button("Save", type="submit"),
    hx_post="/api/save",
    # Close modal on success
    hx_on__after_request=f"""
        if (event.detail.successful) {{
            document.getElementById('{InteractionHtmlIds.modal_dialog('my-modal')}').close();
        }}
    """
)
```

## Server-Side Responses

When updating modal content via HTMX, return just the content:

```python
@app.get("/api/details/{id}")
def get_details(id: str):
    # Load data
    details = load_details(id)
    
    # Return just the content (will replace content area)
    return Div(
        H2(details["title"]),
        P(details["description"]),
        # Include ID to match target
        id=InteractionHtmlIds.modal_dialog_content("details")
    )
```

To show a modal from server-side response, use OOB swap:

```python
@app.post("/api/trigger-action")
def trigger_action():
    # Perform action
    result = do_something()
    
    # Create modal with auto-show
    result_modal = ModalDialog(
        modal_id="result",
        content=Div(
            H2("Success!"),
            P(f"Action completed: {result}")
        ),
        auto_show=True
    )
    
    # Add OOB swap to inject modal into page
    result_modal.attrs['hx-swap-oob'] = 'beforeend:body'
    
    return Div(
        P("Action completed"),  # Main response
        result_modal  # OOB modal
    )
```

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()